In [5]:
# homework/module5_project.py
# ATMS 523 - Module 5 Project
# Robust loader + Baseline + Linear + Polynomial CV + Tuned Random Forest (Windows-friendly)
# Outputs a summary table and saves it to homework/module5_results_summary.csv

from pathlib import Path
import re
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from joblib import parallel_backend


# ----------------------------- Helpers -----------------------------
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def report(name, y_tr, yhat_tr, y_te, yhat_te):
    r2_tr, r2_te = r2_score(y_tr, yhat_tr), r2_score(y_te, yhat_te)
    rmse_tr, rmse_te = rmse(y_tr, yhat_tr), rmse(y_te, yhat_te)
    print(f"{name:>24} | R^2 train: {r2_tr:7.4f} | R^2 test: {r2_te:7.4f} | "
          f"RMSE train: {rmse_tr:9.4f} | RMSE test: {rmse_te:9.4f}")
    return {"model": name, "r2_train": r2_tr, "r2_test": r2_te,
            "rmse_train": rmse_tr, "rmse_test": rmse_te}

def baseline_R_from_Zh_dBZ(zh_dbz_series: pd.Series) -> np.ndarray:
    """
    Baseline Z–R relation:
        Z = 200 * R^1.6, with Z = 10^(dBZ/10)
        => R_hat = ((10^(dBZ/10)) / 200)^(1/1.6)
    """
    Z_linear = 10 ** (zh_dbz_series.values / 10.0)
    R_hat = (Z_linear / 200.0) ** (1.0 / 1.6)
    return R_hat

def norm_key(s: str) -> str:
    """Normalize header text for flexible matching."""
    s = str(s).strip()
    s = re.sub(r"\(.*?\)", "", s)          # remove units in parentheses
    s = s.replace("%", "")
    s = re.sub(r"[\s_\-\/]+", "", s)       # remove spaces/underscores/hyphens/slashes
    s = s.lower()
    s = re.sub(r"[^a-z0-9]", "", s)
    return s


# ----------------------------- Load Data (Robust) -----------------------------
DATA_PATH = Path("homework/radar_parameters.csv")
df = pd.read_csv(DATA_PATH)

# Drop obvious auto-index column if present
df = df.drop(columns=["Unnamed: 0"], errors="ignore")

print("Raw columns in CSV:", list(df.columns))

# Build normalized lookup for present columns
present_norm = {norm_key(c): c for c in df.columns}

# Map common variants to canonical names
alias_map = {
    "Zh":  ["zh", "z", "dbz", "zhh", "zhhh", "reflectivity", "z_h", "z_hh", "zhdbz"],
    "Zdr": ["zdr", "zdrdb", "z_dr"],
    "Ldr": ["ldr", "l_dr"],
    "Kdp": ["kdp", "k_dp", "kdpdegkm1", "kdpdegkm"],
    "Ah":  ["ah", "a_h", "attn", "specificattenuation", "ahh", "specattn", "ahdbzkm"],
    # Treat ADR as ADP for this assignment dataset
    "Adp": ["adp", "a_dp", "diffattenuation", "diffattn", "adr", "a_dr", "adrdbkm", "adpdbkm"],
    "R":   ["r", "rainrate", "rain_rate", "rr", "rain", "preciprate", "rmmhr"],
}

# Choose the best matching column for each canonical name
chosen = {}
for canon, variants in alias_map.items():
    found = None
    for v in variants:
        if v in present_norm:
            found = present_norm[v]
            break
    if not found and canon in df.columns:
        found = canon
    if found:
        chosen[canon] = found

print("Matched columns:", chosen)

required = ["Zh", "Zdr", "Ldr", "Kdp", "Ah", "Adp", "R"]
missing = [c for c in required if c not in chosen]
if missing:
    raise KeyError(
        "Could not find these required columns in the CSV after normalization: "
        f"{missing}\nRaw columns were: {list(df.columns)}\n"
        "If your file truly lacks a column, remove it from `required` & `X` below "
        "or update alias_map with the exact header."
    )

# Rename to canonical names and coerce numeric
df = df.rename(columns={v: k for k, v in chosen.items()})
for c in required:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Drop rows with any NaNs in the used columns
df = df[required].dropna().reset_index(drop=True)

# Feature/target split
X = df[["Zh", "Zdr", "Ldr", "Kdp", "Ah", "Adp"]].copy()
y = df["R"].astype(float).values
X = X.astype(np.float32)
y = y.astype(np.float32)


# ----------------------------- Train/Test Split -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, shuffle=True
)


# ----------------------------- Baseline (Z–R) -----------------------------
yhat_train_base = baseline_R_from_Zh_dBZ(X_train["Zh"])
yhat_test_base  = baseline_R_from_Zh_dBZ(X_test["Zh"])

results = []
results.append(report("Baseline Z–R", y_train, yhat_train_base, y_test, yhat_test_base))


# ----------------------------- Linear Regression -----------------------------
linreg = Pipeline(steps=[
    ("scaler", StandardScaler()),      # optional but consistent across models
    ("lr", LinearRegression())
])
linreg.fit(X_train, y_train)

yhat_train_lr = linreg.predict(X_train)
yhat_test_lr  = linreg.predict(X_test)
results.append(report("Linear Regression", y_train, yhat_train_lr, y_test, yhat_test_lr))


# ----------------------------- Polynomial Regression (0–9), 7-fold CV -----------------------------
from joblib import Memory
cache_dir = Path("homework/.sk_cache"); cache_dir.mkdir(parents=True, exist_ok=True)
memory = Memory(cache_dir, verbose=0)

# cast to float32 to reduce memory pressure
X_train32 = X_train.astype(np.float32)
X_test32  = X_test.astype(np.float32)
y_train32 = y_train.astype(np.float32)
y_test32  = y_test.astype(np.float32)

poly_pipe = Pipeline(steps=[
    ("poly", PolynomialFeatures(include_bias=True)),
    ("scaler", StandardScaler(with_mean=True, with_std=True)),
    ("lr", LinearRegression())
], memory=memory)

cv = KFold(n_splits=7, shuffle=True, random_state=42)
param_grid_poly = {"poly__degree": list(range(0, 10))}

grid_poly = GridSearchCV(
    estimator=poly_pipe,
    param_grid=param_grid_poly,
    scoring="r2",
    cv=cv,
    n_jobs=1,      # <-- single-core to avoid Windows worker crashes
    refit=True,
    verbose=1,
    pre_dispatch="1*n_jobs",
    return_train_score=False,
)

grid_poly.fit(X_train32, y_train32)

best_poly = grid_poly.best_estimator_
best_deg = grid_poly.best_params_["poly__degree"]

yhat_train_poly = best_poly.predict(X_train32)
yhat_test_poly  = best_poly.predict(X_test32)
results.append(report(f"Polynomial (deg={best_deg})", y_train32, yhat_train_poly, y_test32, yhat_test_poly))



# ----------------------------- Random Forest (tuned), 7-fold CV (Windows-friendly) -----------------------------
# Use single-core everywhere to avoid Windows pickling + paging-file errors
rf = RandomForestRegressor(random_state=42, n_jobs=1)  # single-core inside each RF

param_grid_rf = {
    "bootstrap": [True, False],
    "max_depth": [10, 100],
    "max_features": ["sqrt", 1.0],
    "min_samples_leaf": [1, 4],
    "min_samples_split": [2, 10],
    "n_estimators": [200, 1000],
}

grid_rf = GridSearchCV(
    estimator=rf,
    param_grid=param_grid_rf,
    scoring="r2",
    cv=cv,            # 7-fold CV as specified
    n_jobs=1,         # single-core grid search to avoid loky issues on Windows
    refit=True,
    verbose=1,
    pre_dispatch="1*n_jobs",
    return_train_score=False,
)

# With n_jobs=1, this is effectively serial; using threading backend is harmless
with parallel_backend("threading", n_jobs=1):
    grid_rf.fit(X_train, y_train)

best_rf = grid_rf.best_estimator_

yhat_train_rf = best_rf.predict(X_train)
yhat_test_rf  = best_rf.predict(X_test)
results.append(report("Random Forest (tuned)", y_train, yhat_train_rf, y_test, yhat_test_rf))


# ----------------------------- Summary -----------------------------
print("\n=== Summary ===")
summary = pd.DataFrame(results)
print(summary.to_string(index=False))

# Save for your README
out_path = Path("homework/module5_results_summary.csv")
out_path.parent.mkdir(parents=True, exist_ok=True)
summary.to_csv(out_path, index=False)

# Optional: print best model details
print("\nBest Polynomial degree:", best_deg)
print("Best RF params:", grid_rf.best_params_)




Raw columns in CSV: ['Zh (dBZ)', 'Zdr (dB)', 'Ldr (dB)', 'Kdp (deg km-1)', 'Ah (dBZ/km)', 'Adr (dB/km)', 'R (mm/hr)']
Matched columns: {'Zh': 'Zh (dBZ)', 'Zdr': 'Zdr (dB)', 'Ldr': 'Ldr (dB)', 'Kdp': 'Kdp (deg km-1)', 'Ah': 'Ah (dBZ/km)', 'Adp': 'Adr (dB/km)', 'R': 'R (mm/hr)'}
            Baseline Z–R | R^2 train:  0.2756 | R^2 test:  0.3566 | RMSE train:    7.1439 | RMSE test:    7.1893
       Linear Regression | R^2 train:  0.9879 | R^2 test:  0.9891 | RMSE train:    0.9229 | RMSE test:    0.9358
Fitting 7 folds for each of 10 candidates, totalling 70 fits
      Polynomial (deg=3) | R^2 train:  0.9980 | R^2 test:  0.9980 | RMSE train:    0.3742 | RMSE test:    0.4054
Fitting 7 folds for each of 64 candidates, totalling 448 fits


c:\Users\tnaut\anaconda3\envs\xarray-climate\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
14 fits failed out of a total of 448.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
14 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\tnaut\anaconda3\envs\xarray-climate\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\tnaut\anaconda3\envs\xarray-climate\Lib\site-packages\sklearn\base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\tnaut\anaconda3\envs\xarray-climate\Lib\s

   Random Forest (tuned) | R^2 train:  0.9974 | R^2 test:  0.9879 | RMSE train:    0.4268 | RMSE test:    0.9861

=== Summary ===
                model  r2_train  r2_test  rmse_train  rmse_test
         Baseline Z–R  0.275551 0.356643    7.143950   7.189316
    Linear Regression  0.987909 0.989099    0.922940   0.935812
   Polynomial (deg=3)  0.998013 0.997954    0.374167   0.405398
Random Forest (tuned)  0.997415 0.987897    0.426760   0.986075

Best Polynomial degree: 3
Best RF params: {'bootstrap': True, 'max_depth': 100, 'max_features': 1.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
